In [5]:
import sqlite3
import pandas as pd

In [3]:
with sqlite3.connect('surge.db') as conn:
    c = conn.cursor()
    c.execute('select data from requests')
    data = c.fetchall()

In [10]:
import json
from collections import defaultdict
from tqdm.auto import tqdm

keys = defaultdict(set)
for one in tqdm(data):
    for k, v in json.loads(one[0]).items():
        keys[k].add(type(v))

  0%|          | 0/597328 [00:00<?, ?it/s]

In [11]:
keys

defaultdict(set,
            {'id': {int},
             'inMaxSpeed': {int},
             'proxyMode': {bool},
             'notes': {list},
             'inCurrentSpeed': {int},
             'failed': {bool},
             'status': {str},
             'outCurrentSpeed': {int},
             'completed': {bool},
             'sourcePort': {int},
             'completedDate': {float, int},
             'outBytes': {int},
             'sourceAddress': {str},
             'remoteClientPhysicalAddress': {str},
             'policyName': {str},
             'inBytes': {int},
             'method': {str},
             'pid': {int},
             'replica': {bool},
             'rule': {str},
             'startDate': {float, int},
             'streamHasResponseBody': {bool},
             'setupCompletedDate': {float, int},
             'URL': {str},
             'outMaxSpeed': {int},
             'modified': {bool},
             'engineIdentifier': {int},
             'rejected': {bool},
    

In [19]:
with sqlite3.connect('surge.db') as conn:
    sql_query = '''
    select
        json_extract(data, '$.sourceAddress') as sourceAddress,
        json_extract(data, '$.remoteClientPhysicalAddress') as remoteClientPhysicalAddress,
        json_extract(data, '$.policyName') as policyName,
        json_extract(data, '$.status') as status,
        json_extract(data, '$.startDate') as startDate,
        json_extract(data, '$.completedDate') as completedDate,
        json_extract(data, '$.URL') as url,
        json_extract(data, '$.inBytes') as inBytes,
        json_extract(data, '$.outBytes') as outBytes,
        json_extract(data, '$.inMaxSpeed') as inMaxSpeed,
        json_extract(data, '$.outMaxSpeed') as outMaxSpeed
    from requests
    where policyName != 'DIRECT'
    '''
    df = pd.read_sql_query(sql_query, conn)

In [20]:
df['total_MiB'] = (df['inBytes'] + df['outBytes']) / 1024 / 1024

In [21]:
df.groupby(['sourceAddress', 'url', 'policyName'])['total_MiB'].sum().sort_values(ascending=False)

sourceAddress  url                           policyName       
192.168.5.210  cdn.staging.boolv.video:443   🇭🇰 Hong Kong 06      5823.464231
192.168.5.58   images.pexels.com:443         🇭🇰 Hong Kong 06      1409.414956
192.168.5.72   gateway-pre.boolv.tech:443    🇭🇰 Hong Kong 06      1077.854662
192.168.5.214  v1.pinimg.com:443             🇭🇰 Hong Kong 06       355.431819
192.168.5.72   cms-artifacts.artlist.io:443  🇭🇰 Hong Kong 06       290.864006
                                                                     ...     
192.168.5.203  104.193.88.123:80             🇺🇸 USA Seattle 02       0.000000
               104.193.88.77:80              🇺🇸 USA Seattle 02       0.000000
192.168.5.52   101.32.104.226:443            🇭🇰 Hong Kong 06         0.000000
192.168.5.214  www.projectbaseline.com:80    🇭🇰 Hong Kong 06         0.000000
192.168.5.77   8.8.8.8:65535 (Port Map)      🇭🇰 Hong Kong 06         0.000000
Name: total_MiB, Length: 4985, dtype: float64